# Analysis of cooperating algorithms 
### (not finalized)

In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [3]:
filenames = ["data/run" + str(i) + "/loop-1-coop-strategy-2-splicing-data.dat" for i in range(1, 6)]

In [4]:
filenames

['data/run1/loop-1-coop-strategy-2-splicing-data.dat',
 'data/run2/loop-1-coop-strategy-2-splicing-data.dat',
 'data/run3/loop-1-coop-strategy-2-splicing-data.dat',
 'data/run4/loop-1-coop-strategy-2-splicing-data.dat',
 'data/run5/loop-1-coop-strategy-2-splicing-data.dat']

In [10]:
# Create an array of dataframe for each optimization run
dfs_orig = [pd.read_csv(filename) for filename in filenames]

In [6]:
# Code to add RUNID column to each dataframe
def add_run_id(df, id):    
    df['RUNID'] = id          
    columns = df.columns.tolist()
    return df[columns[-1:] + columns[:-1]]

def add_run_ids(dfs):
    for i, df in enumerate(dfs):
        print(i)
        dfs[i] = add_run_id(df, i+1)
    return dfs

In [7]:
# add the run'ids
dfs = add_run_ids(dfs_orig)

0
1
2
3
4


In [8]:
# concatenate the dataframes
dfs = pd.concat(dfs)

In [9]:
dfs.describe()

,RUNID,Iter,Solution,F1(X),F2(X),X10001,X10002,X10003,X10004,X10005,...,X20040,X20041,X20042,X20043,X20044,X20045,X20046,X20047,X20048,Unnamed: 100
count,57000.000000,57000.000000,57000.000000,5.700000e+04,5.700000e+04,57000.000000,57000.000000,57000.000000,57000.000000,57000.000000,...,57000.000000,57000.000000,57000.000000,57000.000000,57000.000000,57000.000000,57000.000000,57000.000000,57000.000000,0.0
mean,2.927895,196.213684,15.500000,2.820612e+09,2.378301e+09,0.440364,0.783269,0.393659,0.410029,0.568030,...,0.485894,0.588381,0.394463,0.419797,0.633460,0.539816,0.308391,0.493190,0.340727,NaN
std,1.318528,118.558087,8.655517,4.664995e+09,4.444799e+09,0.297059,0.198429,0.217880,0.204507,0.225421,...,0.209462,0.200627,0.117920,0.270509,0.356712,0.275071,0.187164,0.227647,0.187184,NaN
min,1.000000,1.000000,1.000000,-1.368161e+10,-2.445000e+10,0.087900,0.039200,0.050496,0.014646,0.258047,...,0.031287,0.095805,0.014751,0.065685,0.011726,0.099932,0.006912,0.183189,0.085726,NaN
25%,2.000000,95.750000,8.000000,-2.649105e+08,-2.656276e+08,0.224257,0.648023,0.220385,0.239438,0.314107,...,0.203241,0.346731,0.289551,0.241552,0.282291,0.335105,0.154060,0.193649,0.192836,NaN
50%,3.000000,190.500000,15.500000,-1.708915e+08,-1.855513e+08,0.298925,0.888705,0.309744,0.430036,0.634564,...,0.594172,0.578712,0.387601,0.353809,0.821594,0.469171,0.282305,0.494883,0.217916,NaN
75%,4.000000,287.250000,23.000000,1.000000e+10,1.000000e+10,0.614913,0.917465,0.518971,0.546511,0.727487,...,0.603386,0.724502,0.473351,0.719179,0.848506,0.890648,0.421914,0.705883,0.585492,NaN
max,5.000000,455.000000,30.000000,1.000000e+10,1.000000e+10,0.991644,0.991417,0.855532,0.986539,0.980144,...,0.892341,0.998579,0.667000,0.958861,0.991203,0.989373,0.797815,0.971690,0.845479,NaN


In [86]:
df1 = dfs[dfs.RUNID==1]

In [87]:
df1

,RUNID,OBJF,X1,X2,X3,X4,X5,X6,X7,X8,...,X39,X40,X41,X42,X43,X44,X45,X46,X47,X48
0,1,-8.501940e+07,0.332800,0.039200,0.305900,0.661700,0.946500,0.848600,0.267500,0.540500,...,0.378800,0.877300,0.922800,0.667000,0.453200,0.537700,0.848500,0.334500,0.752500,0.724000
1,1,-1.430508e+08,0.332800,0.039200,0.305900,0.661700,0.946500,0.848600,0.267500,0.540500,...,0.659889,0.052445,0.107059,0.584993,0.758935,0.172929,0.078109,0.057551,0.356724,0.779963
2,1,-2.140565e+08,0.332800,0.039200,0.305900,0.661700,0.946500,0.848600,0.267500,0.540500,...,0.232399,0.361612,0.277985,0.049247,0.664464,0.264273,0.669980,0.777562,0.504592,0.491920
3,1,-1.807461e+08,0.065541,0.027816,0.152155,0.333449,0.070106,0.208748,0.626339,0.433911,...,0.378800,0.877300,0.922800,0.667000,0.453200,0.537700,0.848500,0.334500,0.752500,0.724000
4,1,-7.503419e+07,0.924193,0.390793,0.662870,0.727935,0.465852,0.043654,0.300683,0.492993,...,0.378800,0.877300,0.922800,0.667000,0.453200,0.537700,0.848500,0.334500,0.752500,0.724000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9925,1,-2.518528e+08,0.469574,0.598363,0.700509,0.405189,0.415218,0.736647,0.482433,0.495667,...,0.138462,0.699329,0.836555,0.643677,0.719179,0.925848,0.352587,0.421914,0.185997,0.143542
9926,1,1.000000e+10,0.614913,0.599139,0.524158,0.410429,0.468654,0.562105,0.787389,0.383049,...,0.396884,0.612285,0.058551,0.682434,0.545007,0.628448,0.848684,0.732760,0.750613,0.747963
9927,1,-2.873171e+08,0.614913,0.599139,0.524158,0.410429,0.468654,0.562105,0.787389,0.383049,...,0.338451,0.843530,0.350929,0.638178,0.739094,0.295605,0.309047,0.315294,0.230848,0.384050
9928,1,-1.928092e+08,0.614913,0.599139,0.524158,0.410429,0.468654,0.562105,0.787389,0.383049,...,0.366230,0.895419,0.557826,0.026380,0.172308,0.585042,0.488261,0.317220,0.763822,0.399433


In [ ]:
# Fill in the line below: get names of columns with missing values
columns_with_missing_values = [col for col in X_train.columns if X_train[col].isnull().any()] # Your code here

# Fill in the lines below: drop columns in training and validation data
reduced_X_train = X_train.drop(columns_with_missing_values, axis=1)
reduced_X_valid = X_valid.drop(columns_with_missing_values, axis=1)

In [ ]:
from sklearn.impute import SimpleImputer

# Fill in the lines below: imputation
my_imputer=SimpleImputer() # Your code here
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# Fill in the lines below: imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

In [ ]:
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

In [ ]:
# Get list of categorical variables
s = (X_train.dtypes == 'object')
object_cols = list(s[s].index)

print("Categorical variables:")
print(object_cols)

In [ ]:
# drop categorical variables
drop_X_train = X_train.select_dtypes(exclude=['object'])
drop_X_valid = X_valid.select_dtypes(exclude=['object'])

print("MAE from Approach 1 (Drop categorical variables):")
print(score_dataset(drop_X_train, drop_X_valid, y_train, y_valid))

In [ ]:
# OrdinalEncoding
from sklearn.preprocessing import OrdinalEncoder

# Make copy to avoid changing original data 
label_X_train = X_train.copy()
label_X_valid = X_valid.copy()

# Apply ordinal encoder to each column with categorical data
ordinal_encoder = OrdinalEncoder()
label_X_train[object_cols] = ordinal_encoder.fit_transform(X_train[object_cols])
label_X_valid[object_cols] = ordinal_encoder.transform(X_valid[object_cols])

print("MAE from Approach 2 (Ordinal Encoding):") 
print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))

In [ ]:
# OneHotEncoder
from sklearn.preprocessing import OneHotEncoder

# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[object_cols]))

# One-hot encoding removed index; put it back
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

print("MAE from Approach 3 (One-Hot Encoding):") 
print(score_dataset(OH_X_train, OH_X_valid, y_train, y_valid))

In [ ]:
# Categorical columns in the training data
object_cols = [col for col in X_train.columns if X_train[col].dtype == "object"]

# Columns that can be safely ordinal encoded
good_label_cols = [col for col in object_cols if 
                   set(X_valid[col]).issubset(set(X_train[col]))]
        
# Problematic columns that will be dropped from the dataset
bad_label_cols = list(set(object_cols)-set(good_label_cols))
        
print('Categorical columns that will be ordinal encoded:', good_label_cols)
print('\nCategorical columns that will be dropped from the dataset:', bad_label_cols)

In [ ]:
# Get number of unique entries in each column with categorical data
object_nunique = list(map(lambda col: X_train[col].nunique(), object_cols))
d = dict(zip(object_cols, object_nunique))

# Print number of unique entries by column, in ascending order
sorted(d.items(), key=lambda x: x[1])

In [ ]:
# Columns that will be one-hot encoded
low_cardinality_cols = [col for col in object_cols if X_train[col].nunique() < 10]

# Columns that will be dropped from the dataset
high_cardinality_cols = list(set(object_cols)-set(low_cardinality_cols))

print('Categorical columns that will be one-hot encoded:', low_cardinality_cols)
print('\nCategorical columns that will be dropped from the dataset:', high_cardinality_cols)